Tiles als Dateien schreiben (mit Overlap) und Metadaten merken

In [1]:
import os
import cv2

IMG_PATH = "/home/gast/Khanh/Repositories/yolov12/test_dji_rgb/ausgabe_ndvi_20_DJI_RGB.tif"
TILE = 640
OVERLAP = 60
STEP = TILE - OVERLAP

TILES_DIR = "tiles_ndvi/ndvi_20_tiles"
os.makedirs(TILES_DIR, exist_ok=True)

img = cv2.imread(IMG_PATH)
H, W = img.shape[:2]

tile_infos = []  # (tile_path, x0, y0, x1, y1)

tile_id = 0
for y0 in range(0, H, STEP):
    for x0 in range(0, W, STEP):
        y1 = min(y0 + TILE, H)
        x1 = min(x0 + TILE, W)
        patch = img[y0:y1, x0:x1]

        tile_path = os.path.join(TILES_DIR, f"tile_{tile_id:05d}_x{x0}_y{y0}.tif")
        cv2.imwrite(tile_path, patch)

        tile_infos.append((tile_path, x0, y0, x1, y1))
        tile_id += 1

print("Tiles:", len(tile_infos))


Tiles: 30


Pro Tile predicten

In [ ]:
import numpy as np
from ultralytics import YOLO

model = YOLO("/home/gast/Khanh/Repositories/yolov12/runs/segment/yolo11m-seg/weights/best.pt")

def mask_to_crop(mask_bool):
    ys, xs = np.where(mask_bool)
    if len(xs) == 0:
        return None
    x0, x1 = xs.min(), xs.max() + 1
    y0, y1 = ys.min(), ys.max() + 1
    crop = mask_bool[y0:y1, x0:x1].astype(np.uint8)  # 0/1
    return x0, y0, x1, y1, crop

instances = []  # globale Instanzen über alle Tiles

for tile_path, x0, y0, x1, y1 in tile_infos:
    r = model(tile_path, 
              imgsz=TILE, 
              conf=0.02, 
              verbose=False,
              )[0]
    if r.masks is None:
        continue

    masks = (r.masks.data.cpu().numpy() > 0.5)      # (N,h,w)
    scores = r.boxes.conf.cpu().numpy()             # (N,)
    clss = r.boxes.cls.cpu().numpy().astype(int)    # (N,)

    for m, s, c in zip(masks, scores, clss):
        crop_info = mask_to_crop(m)
        if crop_info is None:
            continue
        mx0, my0, mx1, my1, crop = crop_info

        # Crop-BBox in globalen Koordinaten (Originalbild)
        gx0 = x0 + mx0
        gy0 = y0 + my0
        gx1 = x0 + mx1
        gy1 = y0 + my1

        area = int(crop.sum())  # Pixel-Fläche (im Originalmaßstab)

        instances.append({
            "cls": int(c),
            "score": float(s),
            "bbox": (gx0, gy0, gx1, gy1),
            "crop": crop,
            "area": area
        })

print("Roh-Instanzen (mit Dubletten):", len(instances))


Results saved to /home/gast/Khanh/Repositories/yolov12/predictions/predict
Results saved to /home/gast/Khanh/Repositories/yolov12/predictions/predict2
Results saved to /home/gast/Khanh/Repositories/yolov12/predictions/predict3
Results saved to /home/gast/Khanh/Repositories/yolov12/predictions/predict4
Results saved to /home/gast/Khanh/Repositories/yolov12/predictions/predict5
Results saved to /home/gast/Khanh/Repositories/yolov12/predictions/predict6
Results saved to /home/gast/Khanh/Repositories/yolov12/predictions/predict7
Results saved to /home/gast/Khanh/Repositories/yolov12/predictions/predict8
Results saved to /home/gast/Khanh/Repositories/yolov12/predictions/predict9
Results saved to /home/gast/Khanh/Repositories/yolov12/predictions/predict10
Results saved to /home/gast/Khanh/Repositories/yolov12/predictions/predict11
Results saved to /home/gast/Khanh/Repositories/yolov12/predictions/predict12
Results saved to /home/gast/Khanh/Repositories/yolov12/predictions/predict13
Results s

Mask-IoU berechnen (Basis für globales Deduplizieren)

In [7]:
def intersects(b1, b2):
    x0,y0,x1,y1 = b1
    X0,Y0,X1,Y1 = b2
    return not (x1 <= X0 or X1 <= x0 or y1 <= Y0 or Y1 <= y0)

def mask_iou(a, b):
    ax0, ay0, ax1, ay1 = a["bbox"]
    bx0, by0, bx1, by1 = b["bbox"]

    ix0, iy0 = max(ax0, bx0), max(ay0, by0)
    ix1, iy1 = min(ax1, bx1), min(ay1, by1)
    if ix1 <= ix0 or iy1 <= iy0:
        return 0.0

    A = a["crop"]
    B = b["crop"]

    # Intersection in A coords
    a_x0, a_x1 = ix0 - ax0, ix1 - ax0
    a_y0, a_y1 = iy0 - ay0, iy1 - ay0
    # Intersection in B coords
    b_x0, b_x1 = ix0 - bx0, ix1 - bx0
    b_y0, b_y1 = iy0 - by0, iy1 - by0

    Ai = A[a_y0:a_y1, a_x0:a_x1]
    Bi = B[b_y0:b_y1, b_x0:b_x1]

    inter = int(np.logical_and(Ai, Bi).sum())
    if inter == 0:
        return 0.0

    union = a["area"] + b["area"] - inter
    return inter / union


Globales Deduplizieren

In [8]:
def mask_nms(instances, iou_thr=0.6):
    inst_sorted = sorted(instances, key=lambda d: d["score"], reverse=True)
    kept = []

    for inst in inst_sorted:
        keep = True
        for k in kept:
            if inst["cls"] != k["cls"]:
                continue
            if not intersects(inst["bbox"], k["bbox"]):
                continue
            if mask_iou(inst, k) > iou_thr:
                keep = False
                break
        if keep:
            kept.append(inst)
    return kept

kept = mask_nms(instances, iou_thr=0.6)
print("Finale Instanzen (dedupliziert):", len(kept))


Finale Instanzen (dedupliziert): 122


Anzahl + Fläche pro Instanz speichern

In [ ]:
count = len(kept)
areas_px = [k["area"] for k in kept]

print("Anzahl:", count)
print("Beispiel-Flächen(px):", areas_px[:10])


In [9]:
import csv, os
os.makedirs("pred_csv", exist_ok=True)

OUT_CSV = "pred_csv/big_instances.csv"
with open(OUT_CSV, "w", newline="", encoding="utf-8") as f:
    w = csv.writer(f)
    w.writerow(["instance_id", "cls", "score", "area_px", "x0", "y0", "x1", "y1"])
    for i, k in enumerate(kept):
        x0,y0,x1,y1 = k["bbox"]
        w.writerow([i, k["cls"], k["score"], k["area"], x0, y0, x1, y1])

print("Gespeichert:", OUT_CSV)


Gespeichert: pred_csv/big_instances.csv


Masken als halbtransparentes Overlay füllen

In [10]:
import cv2
import numpy as np

img = cv2.imread("/home/gast/Khanh/Repositories/yolov12/test_dji_rgb/ausgabe_ndvi_20_DJI_RGB.tif")
vis = img.copy()


In [17]:
alpha = 0.45
COLOR = (255, 255, 0)      # BGR: Cyan/Gelblich
BORDER_COLOR = (0, 0, 0)   # Schwarz
BORDER_THICKNESS = 1

for k in kept:
    gx0, gy0, gx1, gy1 = k["bbox"]
    crop = (k["crop"] * 255).astype(np.uint8)  # für findContours

    roi = vis[gy0:gy1, gx0:gx1]
    if roi.size == 0:
        continue

    # ---------- 1) Maske füllen ----------
    mask_bool = crop.astype(bool)
    roi2 = roi.copy()
    roi2[mask_bool] = (
        roi2[mask_bool] * (1 - alpha)
        + np.array(COLOR, dtype=np.uint8) * alpha
    ).astype(np.uint8)
    vis[gy0:gy1, gx0:gx1] = roi2

    # ---------- 2) Umrandung zeichnen ----------
    contours, _ = cv2.findContours(
        crop,
        cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE
    )

    for cnt in contours:
        cnt[:, 0, 0] += gx0
        cnt[:, 0, 1] += gy0
        cv2.drawContours(
            vis,
            [cnt],
            -1,
            BORDER_COLOR,
            BORDER_THICKNESS
        )

cv2.imwrite("ausgabe20.png", vis)
print("Gespeichert: ausgabe20.png")


Gespeichert: ausgabe20.png
